In [5]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.optimize as opt
import yfinance as yf
import datetime

import warnings

import RECH_functions_2 as RECH

from sklearn.metrics import mean_squared_error as mse

import pickle

In [6]:
act_func = RECH.relu

In [11]:
def srn_dgp(parameters, y_start, sigma_start, T, non_lin_func = RECH.relu):
    (alpha, beta, gamma_0, gamma_1, v_1, v_2, b) = parameters
    eps = np.random.normal(0,1,T)
    y_T = np.zeros(T)
    h_T = np.zeros(T)
    omega_T = np.zeros(T)
    sigma_T = np.zeros(T)
    #y_T[0] = y_start
    for t in range(T):
        if t == 0:
            h_T[t] = non_lin_func(v_1 * np.sign(y_start) * y_start**2 + v_2 * sigma_start + b)
            omega_T[t] = gamma_0 + gamma_1 * h_T[t]
            sigma_T[t] = omega_T[t] + alpha * y_start + beta * sigma_start
            y_T[t] = eps[t] * sigma_T[t]**0.5
        else:
            h_T[t] = non_lin_func(v_1 * np.sign(y_T[t-1]) * y_T[t-1]**2 + v_2 * sigma_T[t-1] + b)
            omega_T[t] = gamma_0 + gamma_1 * h_T[t]
            sigma_T[t] = omega_T[t] + alpha * y_T[t-1]**2 + beta * sigma_T[t-1]
            y_T[t] = eps[t] * sigma_T[t]**0.5
    return y_T, sigma_T, h_T, omega_T

def garch_dgp(parameters, y_start, sigma_start, T):
    (omega, alpha, beta) = parameters    
    eps = np.random.normal(0,1,T)
    y_T = np.zeros(T)
    sigma_T = np.zeros(T)
    for t in range(T):
        if t == 0:
            sigma_T[t] = omega + alpha * y_start + beta * sigma_start
            y_T[t] = eps[t] * sigma_T[t]**0.5
        else:
            sigma_T[t] = omega + alpha * y_T[t-1]**2 + beta * sigma_T[t-1]
            y_T[t] = eps[t] * sigma_T[t]**0.5
    return y_T, sigma_T

def lev_garch_dgp(parameters, y_start, sigma_start, T):
    (omega, alpha1, alpha2, beta) = parameters    
    eps = np.random.normal(0,1,T)
    y_T = np.zeros(T)
    sigma_T = np.zeros(T)
    for t in range(T):
        if t == 0:
            sigma_T[t] = omega + alpha1 * y_start + alpha2 * (y_start < 0)*y_start +  beta * sigma_start
            y_T[t] = eps[t] * sigma_T[t]**0.5
        else:
            sigma_T[t] = omega + alpha1 * y_T[t-1]**2 + alpha2 * (y_T[t-1] < 0)*y_T[t-1]**2 + beta * sigma_T[t-1]
            y_T[t] = eps[t] * sigma_T[t]**0.5
    return y_T, sigma_T

def very_non_lin_garch(parameters, y_start, sigma_start, T):
    (omega, alpha1, alpha2, alpha3, alpha4, beta1, beta2) = parameters
    eps = np.random.normal(0,1,T)
    y_T = np.zeros(T)
    sigma_T = np.zeros(T)
    for t in range(T):
        if t == 0:
            sigma_T[t] = omega + alpha1 * y_start + alpha2 * (y_start / (1+y_start)) + alpha3 * (y_start < 0)*y_start + alpha4 * ( (y_start) / (1+ np.exp(- y_start)))  + beta1 * sigma_start + beta2 * (sigma_start / (1+sigma_start))      
            y_T[t] = eps[t] * sigma_T[t]**0.5
        else:
            sigma_T[t] = omega + alpha1 * y_T[t-1]**2 + alpha2 * (y_T[t-1]**2 / (1+y_T[t-1]**2)) + alpha3 * (y_T[t-1] < 0)*y_T[t-1]**2 + alpha4 * ( (y_T[t-1]<0) / (1+ np.exp(- y_T[t-1]**2)))  + beta1 * sigma_T[t-1] + beta2 * (sigma_T[t-1] / (1+sigma_T[t-1]))                      
            y_T[t] = eps[t] * sigma_T[t]**0.5
    return y_T, sigma_T

In [7]:
T = 1000
M = 1000

# 1. SRN GARCH

In [8]:
act_func = RECH.relu

garch_dgp_pars = (0.05, 0.18, 0.8)
srn_dgp_pars = (0.1, 0.8, 0.07, 0.2, -0.3, 0.5, -0.5)

def con_garch(t):
    return (-1)*(t[1] + t[2]) + 0.999
cons_garch = {'type':'ineq', 'fun': con_garch}

def con_rech(t):
    return (-1)*(t[0] + t[1]) + 0.999
cons_rech = {'type':'ineq', 'fun': con_rech}


def con_gjr(t):
    return (-1)*(t[1] + 0.5 * t[2] + t[3]) + 0.999
cons_gjr = {'type':'ineq', 'fun': con_gjr}

In [19]:
srn_start = (0.1, 0.1, 0.1, 0.1 , 0.1, 0.1, 0, 0.1)
srn_bounds = ((0.0001,1), (0.0001,1), (0.0001, 10),  (0.0001, 3) , (-50, 50), (-50, 50), (0,0)  ,(-100,100 ))
srn_par_names = ["alpha", "beta", "gamma_0", "gamma_1", "v_1", "v_2", "w", "b"]
srn_dgp_pars = (0.1, 0.8, 0.07, 0.2, -0.3, 0.5, 0 , -0.5)

In [20]:
srn_frame = pd.DataFrame(columns = srn_par_names, index = ["DGP:", "Starting Values:", "Bounds:"])

In [21]:
srn_frame

,alpha,beta,gamma_0,gamma_1,v_1,v_2,w,b
DGP:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Starting Values:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bounds:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
srn_frame.loc["DGP:"] = srn_dgp_pars
srn_frame.loc["Starting Values:"] = srn_start
srn_frame.loc["Bounds:"] = srn_bounds

In [24]:
srn_frame

,alpha,beta,gamma_0,gamma_1,v_1,v_2,w,b
DGP:,0.1,0.8,0.07,0.2,-0.3,0.5,NA,-0.5
Starting Values:,0.1,0.1,0.1,0.1,0.1,0.1,0,0.1
Bounds:,"(0.0001, 1)","(0.0001, 1)","(0.0001, 10)","(0.0001, 3)","(-50, 50)","(-50, 50)","(0, 0)","(-100, 100)"


In [9]:
srn_dgp_pars

(0.1, 0.8, 0.07, 0.2, -0.3, 0.5, -0.5)

In [14]:
error_fun_num = 2

garch_start = (0.2, 0.1, 0.7)
garch_bounds = ((0.0001,10), (0.0001,1), (0.0001, 1))
garch_par_names = ["omega", "alpha", "beta"]
garch_pars = pd.DataFrame(np.zeros([M, len(garch_par_names)]), columns = garch_par_names)
garch_errors = np.zeros(error_fun_num * M).reshape(M, error_fun_num)

gjr_start = (0.2, 0.1, 0.1, 0.7)
gjr_bounds = ((0.0001,5), (0.0001,1), (0.0001, 1), (0.0001, 1))
gjr_par_names = ["omega", "alpha", "rho", "beta"]
gjr_pars = pd.DataFrame(np.zeros([M, len(gjr_par_names)]), columns = gjr_par_names)
gjr_errors = np.zeros(error_fun_num * M).reshape(M, error_fun_num)

srn_start = (0.1, 0.1, 0.1, 0.1 , 0.1, 0.1, 0, 0.1)
srn_bounds = ((0.0001,1), (0.0001,1), (0.0001, 10),  (0.0001, 3) , (-50, 50), (-50, 50), (0,0)  ,(-100,100 ))
srn_par_names = ["alpha", "beta", "gamma_0", "gamma_1", "v_1", "v_2", "w", "b"]
srn_pars = pd.DataFrame(np.zeros([M, len(srn_par_names)]), columns = srn_par_names)
srn_errors = np.zeros(error_fun_num * M).reshape(M, error_fun_num)

mgu_start = (0.1, 0.8, 0.2, 0.1 , 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1)
mgu_bounds = ((0.0001,1), (0.0001,1), (0.0001, 10),  (0.0001, 3) , (-10, 10), (-10, 10), (-10, 10), (-10,10), (-10, 10) , (-10, 10), (-100,100), (-100,100))
mgu_par_names = ["alpha", "beta", "gamma_0", "gamma_1", "v_11", "v_12", "v_21", "v_22", "w_1", "w_2", "b_h", "b_z"]
mgu_pars = pd.DataFrame(np.zeros([M, len(mgu_par_names)]), columns = mgu_par_names)
mgu_errors = np.zeros(error_fun_num * M).reshape(M, error_fun_num)

gru_start = (0.1, 0.2, 0.2, 0.1 , 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1)
gru_bounds = ((0.0001,1), (0.0001,1), (0.0001, 10),  (0.0001, 3) , (-10, 10), (-10, 10), (-10, 10), (-10,10), (-10, 10) , (-10, 10), (-10,10), (-10, 10) , (-10, 10), (-100,100), (-100,100), (-100,100))
gru_par_names = ["alpha", "beta", "gamma_0", "gamma_1", "v_11", "v_12", "v_21", "v_22", "v_31", "v_32", "mu_1", "mu_2", "mu_3", "b_h", "b_r", "b_z"]
gru_pars = pd.DataFrame(np.zeros([M, len(gru_par_names)]), columns = gru_par_names)
gru_errors = np.zeros(error_fun_num * M).reshape(M, error_fun_num)

lstm_start = (0.1, 0.8, 0.2, 0.1 , 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1)
lstm_bounds = ((0.0001,1), (0.0001,1), (0.0001, 10),  (0.0001, 3) , (-10, 10), (-10, 10), (-10, 10), (-10,10), (-10, 10) , (-10, 10), (-10,10), (-10, 10) , (-10, 10), (-10,10), (-10, 10) , (-10, 10), (-100,100), (-100,100), (-100,100), (-100,100))
lstm_par_names = ["alpha", "beta", "gamma_0", "gamma_1", "v_11", "v_12", "v_21", "v_22", "v_31", "v_32", "v_41", "v_42", "w_1", "w_2", "w_3", "w_4", "b_c", "b_o", "b_i", "b_f"]
lstm_pars = pd.DataFrame(np.zeros([M, len(lstm_par_names)]), columns = lstm_par_names)
lstm_errors = np.zeros(error_fun_num * M).reshape(M, error_fun_num)

model_num = 1 + 1
fits = np.zeros(M * model_num).reshape(M, model_num)


In [15]:
fit_list_lists = []
fits = np.zeros(model_num * T * M).reshape(model_num, T, M)
for m in range(M):
    returns_m = srn_dgp(srn_dgp_pars, 1, 1, T)[0]
    sigmas_m = srn_dgp(srn_dgp_pars, 1, 1, T)[1]
    fits[0,:,m] = sigmas_m

    res_srn = opt.minimize(RECH.SRN_garch_loglike, srn_start, args = (act_func, returns_m),
                          bounds = srn_bounds,
                           method = "SLSQP",
                          options = {"disp": False, "maxiter": 50000},
                            constraints = cons_rech)
    srn_pars.iloc[m] = res_srn.x
    srn_fit = RECH.SRN_garch(res_srn.x, act_func, returns_m)[0]
    srn_errors[m][0] = RECH.mse(srn_fit, sigmas_m); srn_errors[m][1] = RECH.ql_loss(srn_fit, sigmas_m)
    fits[1,:,m] = srn_fit

    #fit_list_lists.append([returns_m, garch_fit, gjr_fit, srn_fit, mgu_fit, gru_fit, lstm_fit])
#fits = pd.DataFrame(columns = ["true", "garch_fit", "gjr_fit", "srn_fit", "mgu_fit", "gru_fit", "lstm_fit"])
